In [ ]:
from manim import *
import numpy as np

class SphericalCoordinateDemonstration(ThreeDScene):
    def construct(self):
        # Base Sphere and Coordinate Setup
        R_BASE = 2.5
        Z_PLANE = -3.5 # Static vector field Z position
        duration = 5.0 # Duration of each phase

        # Initial Mobjects Setup
        initial_coords = R_BASE * np.array([1, 0, 0]) 
        moving_dot = Dot3D(point=initial_coords, radius=0.1, color=YELLOW_A)
        position_vector = Arrow(
            start=ORIGIN,
            end=initial_coords,
            buff=0,
            color=YELLOW,
            stroke_width=6
        )

        # FIX: Initialize custom attributes 'time' and 'current_phase' immediately
        # This ensures the attributes exist when the updater runs for the first time.
        moving_dot.time = 0 
        moving_dot.current_phase = "IDLE" # Set a safe initial state
        
        # Static Mobjects
        ref_sphere = Sphere(
            radius=R_BASE, resolution=(20, 20), color=BLUE_E, 
            fill_opacity=0.4, stroke_opacity=0.4 
        )
        
        # Static Vector Field (Below the Sphere)
        def vector_field_func(point):
            x, y, z = point
            return np.array([-y / 4, x / 4, 0])

        plane = Square(side_length=10).set_opacity(0.1).move_to([0, 0, Z_PLANE])
        vector_field = ArrowVectorField(
            vector_field_func, x_range=[-4, 4, 1.5], y_range=[-4, 4, 1.5], z_range=[Z_PLANE, Z_PLANE, 1],
            opacity=0.7, length_func=lambda norm: 0.5 * norm, color=RED
        )
        
        # Camera & Initial View
        self.set_camera_orientation(phi=70 * DEGREES, theta=20 * DEGREES, gamma=0 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.1) # Start with a small rotation
        
        # Add Mobjects
        self.add(plane, vector_field, ref_sphere, moving_dot, position_vector)

        # Coordinate Text Setup
        coord_text = VGroup(
            MathTex(r"\rho = 0.00"), MathTex(r"\phi = 0.00"), MathTex(r"\theta = 0.00")
        ).scale(0.6).arrange(DOWN, buff=0.2).to_corner(UL).set_color(YELLOW_B)
        self.add_fixed_in_frame_mobjects(coord_text)
        
        # --- CORE ANIMATION FUNCTION ---
        def get_spherical_coords(time, R_base, phase):
            """Calculates spherical coordinates based on time and phase."""
            rho, phi, theta = R_base, 0, PI/2 

            if phase == "RHO_VARY":
                rho = R_base + 1.0 * np.sin(PI * time / duration)
                phi = 0
                theta = PI / 2 
            
            elif phase == "PHI_VARY":
                rho = R_base
                phi = 2 * PI * time / duration
                theta = PI / 2 
            
            elif phase == "THETA_VARY":
                rho = R_base
                phi = PI / 2 
                theta = PI/2 + (PI / 2) * np.sin(PI * time / duration)
            
            elif phase == "IDLE":
                rho = R_base
                phi = PI / 4
                theta = PI / 3

            x = rho * np.sin(theta) * np.cos(phi)
            y = rho * np.sin(theta) * np.sin(phi)
            z = rho * np.cos(theta)
            
            return np.array([x, y, z]), rho, phi, theta

        def update_animation(mob, dt):
            """Updater function attached to the dot."""
            mob.time += dt
            t = mob.time
            
            current_phase = mob.current_phase
            if t > duration: t = duration 

            new_pos, rho, phi, theta = get_spherical_coords(t, R_BASE, current_phase)

            # Update Dot and Vector
            moving_dot.move_to(new_pos)
            position_vector.put_start_and_end_on(ORIGIN, new_pos)

            # Update Sphere size if varying RHO
            if current_phase == "RHO_VARY":
                ref_sphere.become(
                    Sphere(radius=rho, resolution=(20, 20), color=BLUE_E, fill_opacity=0.4, stroke_opacity=0.4)
                )
            # Reset sphere size if leaving RHO phase (only happens during the reset play)
            elif current_phase != "RHO_VARY" and ref_sphere.radius != R_BASE:
                 ref_sphere.become(
                    Sphere(radius=R_BASE, resolution=(20, 20), color=BLUE_E, fill_opacity=0.4, stroke_opacity=0.4)
                )


            # Update Coordinate Text (Highlighting the active variable)
            new_text = VGroup(
                MathTex(r"\rho = " + f"{rho:.2f}", color=YELLOW if current_phase == "RHO_VARY" else YELLOW_B),
                MathTex(r"\phi = " + f"{phi:.2f}", color=YELLOW if current_phase == "PHI_VARY" else YELLOW_B),
                MathTex(r"\theta = " + f"{theta:.2f}", color=YELLOW if current_phase == "THETA_VARY" else YELLOW_B)
            ).scale(0.6).arrange(DOWN, buff=0.2).move_to(coord_text, aligned_edge=UL)
            
            coord_text.become(new_text)

        # Attach the updater once
        moving_dot.add_updater(update_animation)

        # Ensure dot is at its starting 'IDLE' position before the first phase
        self.wait(1) 

        # =====================================================================
        # PHASE 1: VARY RHO (Radius) 🚀
        # =====================================================================
        self.stop_ambient_camera_rotation()
        self.move_camera(
            phi=0 * DEGREES, 
            theta=90 * DEGREES, 
            gamma=0 * DEGREES, 
            run_time=2
        )
        
        # Set phase AFTER camera move
        moving_dot.current_phase = "RHO_VARY"
        moving_dot.time = 0 
        title1 = Text("Phase 1: Varying ρ (Radius)").to_edge(UP).set_color(YELLOW)
        self.add_fixed_in_frame_mobjects(title1)
        
        self.wait(duration)
        self.play(FadeOut(title1), run_time=1)
        
        # Reset sphere to base size and move camera away before next phase
        # The .animate.become() is what requires the sphere to be reset outside the updater loop
        self.play(
            ref_sphere.animate.become(Sphere(radius=R_BASE, resolution=(20, 20), color=BLUE_E, fill_opacity=0.4, stroke_opacity=0.4)), 
            run_time=1.5
        )

        # =====================================================================
        # PHASE 2: VARY PHI (Azimuthal Angle) 🧭
        # =====================================================================
        self.move_camera(
            phi=1 * DEGREES, 
            theta=0 * DEGREES, 
            gamma=0 * DEGREES,
            run_time=2
        )
        self.begin_ambient_camera_rotation(rate=0.1)
        
        moving_dot.current_phase = "PHI_VARY"
        moving_dot.time = 0 
        title2 = Text("Phase 2: Varying φ (Azimuthal Angle in radians)").to_edge(UP).set_color(YELLOW)
        self.add_fixed_in_frame_mobjects(title2)
        
        self.wait(duration) 
        self.play(FadeOut(title2), run_time=1)

        # =====================================================================
        # PHASE 3: VARY THETA (Polar Angle) 📏
        # =====================================================================
        self.stop_ambient_camera_rotation()
        self.move_camera(
            phi=90 * DEGREES, 
            theta=90 * DEGREES, 
            gamma=0 * DEGREES,
            run_time=2
        )
        
        moving_dot.current_phase = "THETA_VARY"
        moving_dot.time = 0 
        title3 = Text("Phase 3: Varying θ (Polar Angle in radians)").to_edge(UP).set_color(YELLOW)
        self.add_fixed_in_frame_mobjects(title3)
        
        self.wait(duration)
        self.play(FadeOut(title3), run_time=1)
        
        # --- CLEANUP ---
        moving_dot.remove_updater(update_animation)
        self.wait(1)

%manim -ql -v warning SphericalCoordinateDemonstration

Manim Community v0.19.0